# Task A: Deep Learning for ECG Heartbeat Classification

We shall evaluate all models on unseen data, the mitbih_test dataset

In [10]:
import torch
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Get current CUDA device index (if available)
if torch.cuda.is_available():
    print("Current CUDA device index:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("No CUDA devices found.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PyTorch version: 2.5.0+cu118
CUDA available: True
Current CUDA device index: 0
CUDA device name: NVIDIA GeForce RTX 3070


In [11]:
import pandas as pd
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os
dataframes = {}
directory_path = 'Heartbeat_Dataset'
all_files = os.listdir(directory_path)

for file in os.listdir(directory_path):
    if file.endswith('.csv'):
        file_path = os.path.join(directory_path, file)
        # Remove the .csv extension for the DataFrame name
        df_name = os.path.splitext(file)[0]
        dataframes[df_name] = pd.read_csv(file_path)


In [12]:
print(dataframes.keys())


dict_keys(['mitbih_test', 'mitbih_train', 'ptbdb_abnormal', 'ptbdb_normal'])


In [13]:
test_df = dataframes['mitbih_test']

In [14]:
# x_data = data_df.iloc[:, 2:]
# y_label = data_df[['type']]


labels = {
    0.0: "N",
    1.0: "S",
    2.0: "V",
    3.0: "F",
    4.0: "Q"
}

test_df.iloc[:, -1] = test_df.iloc[:, -1].replace(labels)

# Now get the value counts for the renamed last column
train_counts = test_df.iloc[:, -1].value_counts()

# Print the counts
print(train_counts)

0.000000000000000000e+00.65
N    18117
Q     1608
V     1448
S      556
F      162
Name: count, dtype: int64


C:\Users\randa\AppData\Local\Temp\ipykernel_58680\251356151.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        N
1        N
2        N
3        N
4        N
        ..
21886    Q
21887    Q
21888    Q
21889    Q
21890    Q
Name: 0.000000000000000000e+00.65, Length: 21891, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_df.iloc[:, -1] = test_df.iloc[:, -1].replace(labels)


In [15]:
x_data = test_df.iloc[:,:187]
y_label = test_df.iloc[:,-1]

In [16]:
y_label.value_counts()

0.000000000000000000e+00.65
N    18117
Q     1608
V     1448
S      556
F      162
Name: count, dtype: int64

# Loading model

In [17]:
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from common_utils import CNN1D_2


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_label)  # Encode labels without replacement

model_path = "./model/cnn_model2.pth"
num_classes = len(label_encoder.classes_)
loaded_model = CNN1D_2(num_classes).to(device)  # Reinitialize model with the same architecture
loaded_model.load_state_dict(torch.load(model_path, weights_only=True))
print("Model loaded successfully.")

Model loaded successfully.


In [18]:
# Assuming you have a DataLoader for your test dataset
y = label_encoder.fit_transform(y_label.values.ravel())
y = np.eye(num_classes)[y] 

X = x_data
X_test = np.expand_dims(X, axis=1)    # Change axis from 2 to 1
X_test_tensor = torch.tensor(X_test).float()    # Ensure the data is of float type
y_test_tensor = torch.tensor(y).long()   

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

loaded_model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    correct_test = 0
    total_test = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU if available
        outputs = loaded_model(inputs)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels.argmax(dim=1)).sum().item()

    accuracy = correct_test / total_test  # Calculate test accuracy
    print(f'Test Accuracy: {accuracy:.4f}')


Test Accuracy: 0.9830
